In [1]:
# Having jupiter comunicating with django
import setup_django
setup_django.init()

In [8]:
import pandas as pd
from django.conf import settings
from playlists.models import MovieProxy
from ratings.models import Rating


LINKS_SMALL_CSV = settings.DATA_DIR / 'links_small.csv'
LINKS_SMALL_CSV.exists()

True

In [73]:
qs = Rating.objects.all()
missing_movie_ids = []
for instance in qs:
    if instance.content_object is None:
        missing_movie_ids.append(instance.object_id)

        
_total = len(missing_movie_ids)
total_missing = list(set(missing_movie_ids))


In [74]:
print(len(total_missing), _total, qs.count())

272 5350 100004


In [58]:
links_df = pd.read_csv(LINKS_SMALL_CSV)
links_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [59]:
ms_df = links_df.copy()[links_df.movieId.isin(total_missing)]
ms_df.head()

,movieId,imdbId,tmdbId
1,2,113497,8844.0
11,12,112896,12110.0
12,13,112453,21032.0
13,14,113987,10858.0
15,16,112641,524.0


In [60]:
ms_df.shape[0] == len(total_missing)

True

In [61]:
def enrich_imdb_col(val):
    val = str(val)
    if len(val) == 7:
        val = f"tt{val}"
        return val
    if len(val) == 6:
        val = f"tt0{val}"
        return val
    if len(val) == 5:
        val = f"tt00{val}"
        return val
    return val

In [62]:
ms_df['tt'] = ms_df['imdbId'].apply(enrich_imdb_col)
# ms_df.to_csv('')
ms_df.head()

,movieId,imdbId,tmdbId,tt
1,2,113497,8844.0,tt0113497
11,12,112896,12110.0,tt0112896
12,13,112453,21032.0,tt0112453
13,14,113987,10858.0,tt0113987
15,16,112641,524.0,tt0112641


In [63]:
MOVIES_CSV = settings.DATA_DIR / 'movies_metadata.csv'
MOVIES_CSV.exists()

True

In [64]:
movies_cols = ['title', 'overview', 'release_date', 'imdb_id']
movies_df = pd.read_csv(MOVIES_CSV, usecols=movies_cols)
movies_df.head()

,imdb_id,overview,release_date,title
0,tt0114709,"Led by Woody, Andy's toys live happily in his ...",1995-10-30,Toy Story
1,tt0113497,When siblings Judy and Peter discover an encha...,1995-12-15,Jumanji
2,tt0113228,A family wedding reignites the ancient feud be...,1995-12-22,Grumpier Old Men
3,tt0114885,"Cheated on, mistreated and stepped on, the wom...",1995-12-22,Waiting to Exhale
4,tt0113041,Just when George Banks has recovered from his ...,1995-02-10,Father of the Bride Part II


In [65]:
missing_movies_df = ms_df.merge(movies_df, left_on='tt', right_on='imdb_id')
missing_movies_df.head()

,movieId,imdbId,tmdbId,tt,imdb_id,overview,release_date,title
0,2,113497,8844.0,tt0113497,tt0113497,When siblings Judy and Peter discover an encha...,1995-12-15,Jumanji
1,12,112896,12110.0,tt0112896,tt0112896,When a lawyer shows up at the vampire's doorst...,1995-12-22,Dracula: Dead and Loving It
2,13,112453,21032.0,tt0112453,tt0112453,An outcast half-wolf risks his life to prevent...,1995-12-22,Balto
3,14,113987,10858.0,tt0113987,tt0113987,An all-star cast powers this epic look at Amer...,1995-12-22,Nixon
4,16,112641,524.0,tt0112641,tt0112641,The life of the gambling paradise – Las Vegas ...,1995-11-22,Casino


In [66]:
missing_movies_df['id'] = missing_movies_df['movieId']
missing_movies_df['id_alt'] = missing_movies_df['tmdbId'].apply(lambda x: str(int(x)))
missing_movies_df.head()

,movieId,imdbId,tmdbId,tt,imdb_id,overview,release_date,title,id,id_alt
0,2,113497,8844.0,tt0113497,tt0113497,When siblings Judy and Peter discover an encha...,1995-12-15,Jumanji,2,8844
1,12,112896,12110.0,tt0112896,tt0112896,When a lawyer shows up at the vampire's doorst...,1995-12-22,Dracula: Dead and Loving It,12,12110
2,13,112453,21032.0,tt0112453,tt0112453,An outcast half-wolf risks his life to prevent...,1995-12-22,Balto,13,21032
3,14,113987,10858.0,tt0113987,tt0113987,An all-star cast powers this epic look at Amer...,1995-12-22,Nixon,14,10858
4,16,112641,524.0,tt0112641,tt0112641,The life of the gambling paradise – Las Vegas ...,1995-11-22,Casino,16,524


In [67]:
final_df = missing_movies_df.copy()[['id', 'id_alt', 'title']]
final_df['id_alt'] = final_df['id_alt'].astype(str)
final_df.head()

,id,id_alt,title
0,2,8844,Jumanji
1,12,12110,Dracula: Dead and Loving It
2,13,21032,Balto
3,14,10858,Nixon
4,16,524,Casino


In [68]:
alt_id_list = final_df['id_alt'].to_list()

In [69]:
#movie_qs = MovieProxy.objects.filter(id__in=alt_id_list)
movie_qs.count()

959

In [70]:
from django.forms.models import model_to_dict

In [71]:
movies_qs = MovieProxy.objects.filter(id__in=alt_id_list)
for obj in movies_qs:
    data = final_df.copy()[final_df['id_alt'] == str(obj.id)]
    if data.shape[0] == 1:
        og_model_data = model_to_dict(obj)
        update_data = data.to_dict('records')[0]
        if obj.title == update_data.get('title'): 
            #print(og_model_data)
            og_model_data['id'] = update_data['id']
            new_model_data = {**og_model_data}
            #print(new_model_data)
            obj.delete()
            #MovieProxy.objects.create(**new_model_data)
            MovieProxy.objects.create(
                id=new_model_data["id"],
                parent=new_model_data["parent"], 
                category=new_model_data["category"], 
                order=new_model_data["order"], 
                title=new_model_data["title"], 
                type=new_model_data["type"], 
                overview=new_model_data["overview"], 
                release_date=new_model_data["release_date"],  
                slug=new_model_data["slug"], 
                video=new_model_data["video"], 
                active=new_model_data["active"], 
                state=new_model_data["state"], 
                publish_timestamp=new_model_data["publish_timestamp"], 
                rating_last_updated=new_model_data["rating_last_updated"], 
                rating_count=new_model_data["rating_count"], 
                rating_avg=new_model_data["rating_avg"], 
            )

In [72]:
from ratings.tasks import task_update_movie_ratings
task_update_movie_ratings()

Rating update took 0:00:00 (0.0009992122650146484s)
